# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [62]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [63]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [64]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [65]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'MBSGNKGALTQ)C AADLLE(NARETE S O/S',
 'calculationPrice': 'close',
 'open': 135,
 'openTime': 1696772148220,
 'openSource': 'iofiacfl',
 'close': 130.68,
 'closeTime': 1691314015963,
 'closeSource': 'ffoiilac',
 'high': 131.38,
 'highTime': 1657703910036,
 'highSource': 'iia5m1eplc duyee nerdt ',
 'low': 133.08,
 'lowTime': 1629745166927,
 'lowSource': 'r1uitneed 5ae yidcl emp',
 'latestPrice': 136.77,
 'latestSource': 'Close',
 'latestTime': 'April 8, 2021',
 'latestUpdate': 1667768227100,
 'latestVolume': 90991167,
 'iexRealtimePrice': 135.93,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1686967674546,
 'delayedPrice': 133.699,
 'delayedPriceTime': 1655829490158,
 'oddLotDelayedPrice': 132.46,
 'oddLotDelayedPriceTime': 1618633885469,
 'extendedPrice': 131.08,
 'extendedChange': 0.5,
 'extendedChangePercent': 0.00397,
 'extendedPriceTime': 1629560379571,
 'previousClose': 130.2,
 'previousVolume': 86732385,
 'change':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [66]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [67]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],index=my_columns
    ),ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,132.3,2246378675411,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],index=my_columns
        ),ignore_index=True
    )
    
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,132.45,40320432376,N/A
1,AAL,23.78,15243879335,N/A
2,AAP,189.20,12102905828,N/A
3,AAPL,136.14,2297067318540,N/A
4,ABBV,110.20,190493259916,N/A
...,...,...,...,...
500,YUM,118.82,34437525115,N/A
501,ZBH,169.94,34969816184,N/A
502,ZBRA,509.60,28072893208,N/A
503,ZION,56.00,9262014417,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [68]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [69]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],index=my_columns
            ),ignore_index=True
        )
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.13,41464384079,N/A
1,AAL,24.00,15719555307,N/A
2,AAP,180.70,12021171636,N/A
3,AAPL,133.67,2291231976051,N/A
4,ABBV,109.90,194497273266,N/A
...,...,...,...,...
500,YUM,119.21,35167012871,N/A
501,ZBH,165.60,35603200057,N/A
502,ZBRA,527.30,27939811505,N/A
503,ZION,55.00,9157099409,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [70]:
#this had been delayed till now, because the number of shares to buy will be based on the personal portfolio which shall be taken as an user input
flag=0
while(flag == 0):
    try:
        portfolio_size = input('Enter the value of your portfolio: ')
        val = float(portfolio_size)
        flag=1
    except ValueError:
        print("That is not a number\nPlease try again")
val

Enter the value of your portfolio: 10000000


10000000.0

In [74]:
#position size implies how much money you are going to invest in each stock
#Now since all the stocks have equal weight in our example, hence say you have a total of 505000 dollars to invest,
#then as all the 505 stocks have equal weight you will invest 505000/505 = 1000 dollars in each stock

position_size = val/len(final_dataframe.index)
print(position_size)

#Now, we need to calculate how many shares of each stock we need to buy to get the above calculated position for each stock

for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe


19801.980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.13,41464384079,144
1,AAL,24.00,15719555307,825
2,AAP,180.70,12021171636,109
3,AAPL,133.67,2291231976051,148
4,ABBV,109.90,194497273266,180
...,...,...,...,...
500,YUM,119.21,35167012871,166
501,ZBH,165.60,35603200057,119
502,ZBRA,527.30,27939811505,37
503,ZION,55.00,9157099409,360


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [85]:
#the class that we are importing the xlsxwriter is from pandas as pandas by default works great with relational data format
#the engine is where we mention the xlsxwriter library

writer  = pd.ExcelWriter('recommended trades.xlsx',engine='xlsxwriter')

#Recommended Trades is the tab of the excel file where we want to save the table
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [86]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)


### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [54]:
# writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18,string_format)
# writer.save()

# writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recommended Trades'].write('B1','Stock Price',string_format)
# writer.sheets['Recommended Trades'].write('C1','Market Capitalization',string_format)
# writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy',string_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [87]:
column_format = {
    'A': ['Ticker',string_format],
    'B': ['Stock Price',dollar_format],
    'C': ['Market Capitalization',dollar_format],
    'D': ['Number of Shares to Buy',integer_format],
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',25,column_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_format[column][0],string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [88]:
writer.save()